# 06 SemanticKernel | 02 Plugin - Semantic Function - File Definition

## Semantic function as part of a Skill (external definition)

In [this notebook](./01-Semantic-PlugIn-Inline.ipynb) a semantic function was defined inline as part of the code. Changing prompts means also re-compiling the application. To de-couple prompts from application code Semantic Kernel provides functionality to import prompts and supporting prompt configuration files from a directory structure. 

Per semantic function two files need to be provided:

- skprompt.txt (prompt definition)
- config.json (LLM call properties)

Additional information: 
- [Overview Semantic Function](./README.md)
- [Serializing semantic functions](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/serializing-semantic-functions?tabs=Csharp)

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)) 

## Step 1: Create Semantic Kernel instance

An instance of Semantic Kernel can be created using the KernelBuilder object provided by the Semantic Kernel SDK. It acts as the centralized point for all .NET functionality that want to interact with Semantic Kernel functionality or concepts.

It abstracts e.g., models from the OpenAI GPT family and can communicate with Azure OpenAI LLM instances as well as LLMs deployed on OpenAI. In the sample we use models deployed on Azure.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta1"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using DotNetEnv;
using Microsoft.SemanticKernel;

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY") ?? "SKIT_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT") ?? "SKIT_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME") ?? "SKIT_CHATCOMPLETION_DEPLOYMENTNAME not found";

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        endpoint: oAiEndpoint, 
        apiKey: oAiApiKey,
        deploymentName: chatCompletionDeploymentName)
    .Build();

Console.WriteLine($"Semantic Kernel initialized...");



#### Expected output:

```
Installed Packages
DotNetEnv, 2.5.0
Microsoft.SemanticKernel, 1.0.0-beta1

Semantic Kernel initialized...
```

## Step 2: Import Semantic Functions

The kernel function `ImportSemanticFunctionsFromDirectory()` can be used to make Semantic Kernel aware of external function definitions. Each external function is defined in two files: 

  - [*skprompt.txt*](./PluginLibrary/CreateSummary/SimpleSummary/skprompt.txt): contains the prompt definition
  - [*config.json*](./PluginLibrary/CreateSummary/SimpleSummary/config.json): contains parameter to be used when calling the LLM  


In [ ]:
using System.IO; 

string pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "PluginLibrary");

string plugInName = "CreateSummary";
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, plugInName);

Console.WriteLine($"Semantic functions imported from directory '{pluginsDirectory}'...");


#### Expected output:

```
Semantic functions imported from directory 'xxxx\docs\07_SemanticKernel\PluginLibrary'...
```

## Step 4: Execute Skill from Plugin

Just like with semantic functions defined inline the kernel function `RunAsync()` is used to execute the external semantic function.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

//Read text from file which should be summarized
string assetsFolder = @"../../assets";
string input = await File.ReadAllTextAsync(Path.Combine(assetsFolder,"docs","06_SemanticKernel", "aci_documentation.txt"));

//Define function which need to be called
string functionName = "SimpleSummary";

//Call Function
ISKFunction skFunction = kernel.Functions.GetFunction(plugInName, functionName);
KernelResult result = await kernel.RunAsync(input, skFunction); 

Console.WriteLine($"Summary: {result.GetValue<string>().Substring(0,50)}...");


#### Expected output:

```
Summary: This article explains how to use the Deploy to Azu...
```

## Next steps

Now that semantic functions have been defined, registered and executed from [inline](./01-Semantic-PlugIn-Inline.ipynb) and external (this Notebook) next steps are:

- [Define, register and execute a native (c#) function](./03_PlugIn_NativeFunction.ipynb)

